In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
! pip install yfinance --upgrade --no-cache-dir


Requirement already up-to-date: yfinance in c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages (0.2.43)


You should consider upgrading via the 'c:\users\saidj\onedrive\documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
# import yfinance as yf

# df = yf.Ticker("USDJPY").history(period="max").reset_index()
# df

In [5]:
# symbols_path = './backbone/data/backtest/symbols/USDCAD_D.csv'
# df = pd.read_csv(symbols_path)
# df

In [75]:
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz

print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
utc_from = datetime(2000, 1, 1, tzinfo=timezone)
utc_to = datetime(2024, 1, 1, tzinfo=timezone)

# Pares de divisas
tickers = ['EURUSD', 'USDHKD']
main_ticker = 'USDHKD'

# DataFrame final
final_df = pd.DataFrame()

for ticker in tickers:
    # Obtener las tasas históricas
    rates = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, utc_from, utc_to)
    
    # Crear DataFrame con las tasas
    df = pd.DataFrame(rates)
    
    # Convertir el tiempo de segundos a formato datetime
    df['time'] = pd.to_datetime(df['time'], unit='s')

    if ticker != main_ticker:
        # Renombrar columnas para los tickers que no son el principal
        df = df.rename(columns={
            'time': f'{ticker}_Date',
            'open': f'{ticker}_Open',
            'high': f'{ticker}_High',
            'low': f'{ticker}_Low',
            'close': f'{ticker}_Close',
            'tick_volume': f'{ticker}_Volume'
        }).set_index(f'{ticker}_Date')

    else:
        # Renombrar columnas para el ticker principal
        df = df.rename(columns={
            'time': 'Date',
            'open': 'Open',
            'high': 'High',
            'low': 'Low',
            'close': 'Close',
            'tick_volume': 'Volume'
        }).set_index('Date')

    # Merge del DataFrame actual con el final
    if final_df.empty:
        final_df = df
    else:
        final_df = pd.merge(
            final_df,
            df,
            left_index=True,
            right_index=True,
            how='inner'
        )

# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

# Mostrar el DataFrame final
print(final_df)


MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
            EURUSD_Open  EURUSD_High  EURUSD_Low  EURUSD_Close  EURUSD_Volume  \
2007-07-10      1.36260      1.37850     1.35940       1.37680           6576   
2007-07-11      1.37670      1.37790     1.37300       1.37440           7524   
2007-07-12      1.37430      1.38000     1.37410       1.37870           6496   
2007-07-13      1.37860      1.38120     1.37620       1.37820           5923   
2007-07-16      1.37800      1.38040     1.37590       1.37720           3728   
...                 ...          ...         ...           ...            ...   
2023-12-25      1.10119      1.10162     1.10119       1.10135            624   
2023-12-26      1.10028      1.10451     1.09998       1.10415          97243   
2023-12-27      1.10414      1.11229     1.10284       1.11043         163798   
2023-12-28      1.11050      1.11396     1.10550       1.10609         181168   
2023-12-29      1.10602  

In [76]:
df = final_df.reset_index().rename(columns={'index':'Date'})

train_start = '2007-01-01'
train_end = '2023-12-01'

test_start = '2003-01-01'
test_end = '2005-01-01'

wfo_start = '2005-01-01'
wfo_end = '2024-01-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


## Test Entry: Monkey Exit

In [77]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


In [90]:

import itertools

class LowerLow(Strategy):
    pip_size = 0.0001
    risk=3
    rsi_threshold=99
    daily_bars_low = 10
    intra_day_bars_low = 5

    
    def init(self):
        self.eur_usd_daily_sma_200 = self.I(
            ta.SMA, self.data.EURUSD_Close, timeperiod=200
        )

        self.usdhkd_daily_sma_200 = self.I(
            ta.SMA, self.data.Close, timeperiod=200
        )


        self.daily_rsi_2 = self.I(
            ta.RSI, self.data.EURUSD_Close, 2
        )

        self.daily_ll, self.daily_hh = self.I(
            ll_hh_indicator, self.data.EURUSD_Close, window=self.daily_bars_low
        )
        
        self.engulfing = self.I(ta.CDLENGULFING, self.data.EURUSD_Open, self.data.EURUSD_High, self.data.EURUSD_Low, self.data.EURUSD_Close)

    
    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            # today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')
            # first_trade = self.trades[0]
            # time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            # time_in_position = time_in_position.days

            if self.position.is_long:
                if self.daily_rsi_2 >= 65:
                    self.position.close()

            # if self.position.is_short:
            #     if self.daily_rsi_2 <= 35 or time_in_position >= 5 or actual_close >= self.daily_sma_200:
            #         self.position.close()
        else:

            actual_rsi = self.daily_rsi_2[-1]


            if self.data.EURUSD_Close[-1] > self.eur_usd_daily_sma_200 and actual_rsi <= 10 and self.daily_ll:
                self.buy(size=self.risk / 100)
                
            # if actual_close < self.daily_sma_200 and cum_rsi >= 90 and self.daily_hh:
            #     self.sell(size=self.risk / 100)


bt_train = Backtest(
    train_data, 
    LowerLow,
    # commission=0.01, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()
bt_train.plot(filename='./RsiBBands.html', resample=False)

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2007-07-10 00:00:00
End                       2023-11-30 00:00:00
Duration                   5987 days 00:00:00
Exposure Time [%]                    9.427527
Equity Final [$]                  15023.72921
Equity Peak [$]                   15105.92187
Return [%]                           0.158195
Buy & Hold Return [%]               -0.090196
Return (Ann.) [%]                    0.009704
Volatility (Ann.) [%]                0.157236
Sharpe Ratio                         0.061717
Sortino Ratio                        0.091227
Calmar Ratio                         0.010425
Max. Drawdown [%]                   -0.930886
Avg. Drawdown [%]                   -0.100756
Max. Drawdown Duration     4501 days 00:00:00
Avg. Drawdown Duration      427 days 00:00:00
# Trades                                   59
Win Rate [%]                        50.847458
Best Trade [%]                       0.276133
Worst Trade [%]                     -0.217469
Avg. Trade [%]                    

In [41]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1732,1632,1637,7.79090,7.78980,-1.90520,-0.000141,2008-04-25 03:00:00,2008-04-25 18:00:00,0 days 15:00:00
1,1732,1639,1643,7.79100,7.78960,-2.42480,-0.000180,2008-04-28 00:00:00,2008-04-28 12:00:00,0 days 12:00:00
2,1729,1944,1968,7.80410,7.80700,5.01410,0.000372,2008-06-19 03:00:00,2008-06-24 03:00:00,5 days 00:00:00
3,1731,2008,2040,7.79680,7.79660,-0.34620,-0.000026,2008-07-01 03:00:00,2008-07-07 03:00:00,6 days 00:00:00
4,1731,2096,2144,7.79690,7.79980,5.01990,0.000372,2008-07-16 03:00:00,2008-07-24 03:00:00,8 days 00:00:00
5,1729,2312,2376,7.80690,7.80810,2.07480,0.000154,2008-08-22 03:00:00,2008-09-03 03:00:00,12 days 00:00:00
6,1731,2408,2409,7.80150,7.79720,-7.44330,-0.000551,2008-09-09 03:00:00,2008-09-09 06:00:00,0 days 03:00:00
7,1730,2410,2413,7.80000,7.79790,-3.63300,-0.000269,2008-09-09 09:00:00,2008-09-09 18:00:00,0 days 09:00:00
8,1730,2414,2418,7.80030,7.79810,-3.80600,-0.000282,2008-09-09 21:00:00,2008-09-10 09:00:00,0 days 12:00:00
9,1729,2419,2423,7.79980,7.79800,-3.11220,-0.000231,2008-09-10 12:00:00,2008-09-11 00:00:00,0 days 12:00:00


In [ ]:
bt_train = Backtest(
    test_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats

## Test Exit: Monkey Entry

In [ ]:

trade_frecuency = stats['# Trades'] / train_data.shape[0]
long_frecuency = stats._trades[stats._trades['Size'] > 0].shape[0] / stats['# Trades']
short_frecuency = stats._trades[stats._trades['Size'] < 0].shape[0] / stats['# Trades']

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]


class VwapExit(Strategy):
    pip_size = 0.0001
    sl_pips = 15
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            if self.position.is_long and self.rsi > 70:
                self.position.close()
                
            elif self.position.is_short and self.rsi < 30:
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    sl = actual_close - self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(sl=sl, size=units)

                else:
                    sl = actual_close + self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(sl=sl, size=units)


metrics = pd.DataFrame()

for x in range(0, 10):
    bt_train = Backtest(
        train_data, 
        VwapExit, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
    )

    # bt_train.plot(filename='./RsiBBands.html', resample=False)

    equity = stats['Equity Final [$]']
    return_ = stats['Return [%]']
    sharpe_ratio = stats['Sharpe Ratio']

    actual_metrics = pd.DataFrame(
        {
            'equity':[equity], 
            'return_':[return_], 
            'sharpe_ratio':[sharpe_ratio], 
        }
    )

    metrics = pd.concat([metrics, actual_metrics])



In [ ]:
metrics.return_.std()

## Full system

In [ ]:
import itertools

class VwapRsiFull(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1
    risk = 1

    n_candles = 7 
    distance = 1

    rsi_up_threshold=70
    rsi_down_threshold=30


    def init(self):
        self.vwap = self.I(
            pandas_ta.vwap, 
            pd.Series(self.data.High, index=self.data.index),
            pd.Series(self.data.Low, index=self.data.index),
            pd.Series(self.data.Close, index=self.data.index),
            pd.Series(self.data.Volume, index=self.data.index),
        ) 

        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        
        self.rsi = self.I(ta.RSI, self.data.Close, 14)


    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            pass

        else: 

            actual_vwap = self.vwap[-1]
            distance_vwap = diff_pips(actual_vwap, actual_close, pip_value=self.pip_size, absolute=True)

            n_candles_under_vwap = True
            n_candles_up_vwap = True

            for x in range(1, self.n_candles):
                if self.data.Close[-x] > self.vwap[-x]:
                    n_candles_under_vwap = False

                if self.data.Close[-x] < self.vwap[-x]:
                    n_candles_up_vwap = False

            if distance_vwap <= self.distance and n_candles_up_vwap: 
                sl = actual_close - self.sl_pips * self.pip_size
                tp = actual_close + self.rr * self.sl_pips * self.pip_size
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl, size=units, tp=tp)

            elif distance_vwap <= self.distance and n_candles_under_vwap: 
                sl = actual_close + self.sl_pips * self.pip_size
                tp = actual_close - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(sl=sl, size=units, tp=tp)


bt_train = Backtest(
    train_data, 
    VwapRsiFull, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)


stats = bt_train.run(rr=2, sl_pips=15)

# stats = bt_train.optimize(
#     distance=[1, 2, 3],
#     n_candles=[6, 8,  10, 12],
#     sl_pips=[5, 8, 12, 15],
#     rsi_up_threshold=[60, 70, 80],
#     rsi_down_threshold=[20, 30, 40],
# )

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


In [ ]:
stats._strategy.__dict__

# WFO

In [ ]:
import pickle
from wfo_utils.utils import walk_forward, plot_full_equity_curve, plot_stats

In [ ]:

lookback_bars = 365 * 24 + 200 * 24
validation_bars = 180 * 24
warmup_bars = 200 * 24 

params = {
    'n': [10],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    LowerLow,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

In [ ]:
for x in range(0, len(stats)):
    plot_stats(df, stats[x], LowerLow, plot=True)

In [ ]:
plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
